In [12]:
#after changes in the script using Jupyter notebook, save, and run this:
get_ipython().system(u'jupyter nbconvert --to=python integrate_CO2flux.ipynb')

[NbConvertApp] Converting notebook integrate_CO2flux.ipynb to python
[NbConvertApp] Writing 5022 bytes to integrate_CO2flux.py


In [9]:
    def integrate_CO2flux(datam,region,areaT,xax,yax):
        """
        this function integrates a monthly 3d field of CO2 flux into an annual mean time-series, over specific regions

        """
      
        import numpy as np
        from annualmean import annualmean
        import matplotlib.pyplot as plt
        import matplotlib.cm as cm
        from netCDF4 import Dataset

        cmap = cm.jet
        cmap.set_bad('white',1.)
        
        nn = np.shape(datam)
        
        nt = nn[0]
        nx=nn[2]
        ny=nn[1]
        
        #print np.shape(yax)
        yax2=yax[np.newaxis,:,np.newaxis]
        #print np.shape(yax2)
        YY = np.tile(yax2,(nt,1,nx))
        
#        print np.shape(YY)
               
        # define indices for regions
        # define areas
        

        maskG = (YY[:] < 90)
        maskN = ((YY[:]>30))
        maskN3060 = ((YY[:]>30)& (YY[:] <= 60))
        maskN60 = (YY[:]>60)
        maskS = (YY[:]<-30)
        maskT = ((YY[:]>-30) & (YY[:] < 30))

        regions = Dataset('/work/ollie/jhauck/reccap/RECCAP2-shared-resources/data/regions/RECCAP2_region_masks_all.nc',mode='r')
        reg_lon = regions['lon'][:]
        reg_lat = regions['lat'][:]
        reg_arc = regions['arctic'][:]
        reg_arc[reg_arc>0]=1 # 1 in Arctic, 0 elsewhere
        reg_arcT = np.tile(reg_arc,(nt,1,1))
        maskArc = (reg_arcT>0)
        

                
#         reg_noarc = regions['arctic'][:]
#         reg_noarc[reg_noarc>0] = -1  # -1 in Arctic, 
#         reg_noarc[reg_noarc==0] = True  #1 elsewhere
#         reg_noarc[reg_noarc<0] = False  # 0 in Arctic, 1 elsewhere
#         reg_noarcT = np.tile(reg_noarc,(nt,1,1))
        maskNoArc = (reg_arcT<1)
        maskN_noArc = ((YY[:]>30) & (reg_arcT<1))

        
        print('shape(reg_arcT)',np.shape(reg_arcT))
        
#         plt.figure()
#         plt.pcolor(reg_lon,reg_lat,np.squeeze(maskNoArc[1,:,:]))#, vmin=-60, vmax=750)
#         plt.colorbar()
#         plt.xlabel('no Arctic')
#         plt.show()
                
        
#            indN = np.where(yax > 30)
#            indS = np.where(yax < -30)
#            indT = np.where((yax>-30) & (yax<30))
            
        # define right region index
        if region == 'global':
            mask = maskG
       # elif self.region == 'pac': #do not filter more, indR should be global/all
       #     indR = glob
        elif region == 'S':
            mask = maskS
        elif region == 'N':
            mask = maskN
        elif region == 'N3060':
            mask = maskN3060
        elif region == 'N60':
            mask = maskN60 # this is the old mask for north of 60N
        elif region == 'Arc':
            mask = maskArc
        elif region == 'glo_noArc':
            mask = maskNoArc
            
        elif region == 'N_noArc':
            mask = maskN_noArc
#            mask[isnan(reg_noarc)]=np.nan
            
        elif region == 'T':
            mask = maskT
        else:
            raise NameError('Region not defined properly')
            
        print('region', region)
        print('shape(mask)',np.shape(mask))
#        print('mask',mask)
        plt.figure()
        plt.pcolor(reg_lon,reg_lat,np.squeeze(mask[1,:,:]))#, vmin=-60, vmax=750)
        plt.colorbar()
        plt.xlabel('mask')
        plt.show()
            
        #unit is molC/m2/s, convert to PgC/yr
        # *12: mol --> gC, *1e-15: g --> Pg, *86400*365.25: /s -> /yr
        conv=12*1e-15 *86400*365.25
 
       # print np.shape(datam)
    
        # initialize array full of NaNs
        datam2 = np.full_like(datam, np.nan)
        
        # fill with values of region to be used, rest stays NaN 
        print(region)
        print(np.shape(datam2))
        print(np.shape(datam))
        print(np.shape(mask))
        
        print(np.max(np.max(np.max(mask))))
        print(np.min(np.min(np.min(mask))))
        print(np.nanmax(np.nanmax(np.nanmax(mask))))
        print(np.nanmin(np.nanmin(np.nanmin(mask))))
        
        datam2[mask] = datam[mask]
        data_m = np.ma.array(datam2, mask=np.isnan(datam2))
    
        tmp2 = data_m * areaT   
    
    # print np.shape(tmp2)
        # sum up and convert units
        #sum first over xax, then yax, keep timesteps (axis0)
        tmp3 = conv*(np.nansum(np.nansum(tmp2,axis=2),axis=1))
      #  print np.shape(tmp3)
        fgco2_global=tmp3
        # annual_mean
        fgco2_global_am=annualmean(tmp3)
      #  print np.shape(fgco2_global_am) 
        
        #########################################   
        ### quickly check fields:
        
    #    lon, lat = np.meshgrid(xax,yax, indexing='ij')

        #print np.shape(np.squeeze(datam[:,indR,:]))
        #graphic models
    #    print "plotting year 2015 December"
    #    fig_mod = plt.figure(1,figsize=(10,10))
    #    plt.pcolor(xax,yax,np.squeeze(data_m[-25,:,:]), vmin=-2e-7, vmax=5e-7, cmap=cmap)#vmin=np.nanmin(x), vmax=np.nanmax(x)) 
    #    plt.colorbar()
    #    plt.show()
        return fgco2_global_am
    
        
    

#test
import numpy as np
from netCDF4 import Dataset
gcb_data = Dataset('../Output/gcb_load.nc')
roeden_spco2_1x1 = gcb_data['roeden_spco2_1x1'][:]

mean = annualmean(roeden_spco2_1x1)
print np.shape(roeden_spco2_1x1), np.shape(mean)